In [19]:
from pathlib import Path

ROOT = Path(r"c:/Users/luciu/OneDrive/WorkStation/momentum-hk")
cleaned = ROOT / "data" / "cleaned"

print(cleaned)  # c:\Users\luciu\OneDrive\WorkStation\momentum-hk\data\cleaned



c:\Users\luciu\OneDrive\WorkStation\momentum-hk\data\cleaned


In [8]:
import yfinance as yf
import pandas as pd
import os
import sys
#from momentumhk.MomentumBacktestClass import MomentumVectorBacktest as MVB
from typing import Iterable, Optional, Tuple, List

In [ ]:
def quarter_end_dates(dates: pd.DatetimeIndex) -> pd.DatetimeIndex:
    """
    Return the trading days that are quarter-ends based on `dates`.
    """
    # Group by (year, quarter), take last date in each group
    q = pd.Series(dates, index=dates)
    grp = q.groupby([dates.year, dates.quarter]).last()
    return pd.DatetimeIndex(grp.values)


def quarterly_rebalance_dates(
    prices: pd.DataFrame,
    warmup_days: int = 252 + 21,  # ~13 months warmup for 12-1 momentum
    start: Optional[pd.Timestamp] = None,
    end: Optional[pd.Timestamp] = None,
) -> pd.DatetimeIndex:
    """
    Compute quarterly rebalance dates (last trading day of each quarter).
    """
    idx = prices.index
    qend = quarter_end_dates(idx)

    if start is not None:
        qend = qend[qend >= pd.Timestamp(start)]
    if end is not None:
        qend = qend[qend <= pd.Timestamp(end)]

    # Apply warmup: drop any rebalance dates earlier than warmup_days into the series
    first_valid = idx[min(len(idx) - 1, warmup_days)]
    qend = qend[qend >= first_valid]
    return qend

In [12]:
dates = pd.date_range(start='2025-01-01', end = '2025-10-05')
dates

DatetimeIndex(['2025-01-01', '2025-01-02', '2025-01-03', '2025-01-04',
               '2025-01-05', '2025-01-06', '2025-01-07', '2025-01-08',
               '2025-01-09', '2025-01-10',
               ...
               '2025-09-26', '2025-09-27', '2025-09-28', '2025-09-29',
               '2025-09-30', '2025-10-01', '2025-10-02', '2025-10-03',
               '2025-10-04', '2025-10-05'],
              dtype='datetime64[ns]', length=278, freq='D')

In [13]:
quarter_end_dates(dates)

DatetimeIndex(['2025-03-31', '2025-06-30', '2025-09-30', '2025-10-05'], dtype='datetime64[ns]', freq=None)

In [18]:
ROOT.resolve()

WindowsPath('C:/Users/luciu/OneDrive/WorkStation/momentum-hk/notebooks')

In [29]:
prices = pd.read_csv(cleaned / "9999.HK.csv", index_col="Date", parse_dates=True)



In [30]:
prices.index


DatetimeIndex(['2020-06-11', '2020-06-12', '2020-06-15', '2020-06-16',
               '2020-06-17', '2020-06-18', '2020-06-19', '2020-06-22',
               '2020-06-23', '2020-06-24',
               ...
               '2025-10-16', '2025-10-17', '2025-10-20', '2025-10-21',
               '2025-10-22', '2025-10-23', '2025-10-24', '2025-10-27',
               '2025-10-28', '2025-10-30'],
              dtype='datetime64[ns]', name='Date', length=1325, freq=None)

In [31]:
quarterly_rebalance_dates(prices=prices)

DatetimeIndex(['2021-09-30', '2021-12-31', '2022-03-31', '2022-06-30',
               '2022-09-30', '2022-12-30', '2023-03-31', '2023-06-30',
               '2023-09-29', '2023-12-29', '2024-03-28', '2024-06-28',
               '2024-09-30', '2024-12-31', '2025-03-31', '2025-06-30',
               '2025-09-30', '2025-10-30'],
              dtype='datetime64[ns]', freq=None)